In [1]:
import pandas as pd
import numpy as np
import os
import csv
import re
import logging
import optparse
import re
import spacy
import dedupe
import pickle
import copy
import json
from unidecode import unidecode
import recordlinkage as rl

In [2]:
import pandas as pd
import os
import csv
import re
import logging
import optparse
import re
import spacy
import dedupe
import pickle
import copy
import json
from unidecode import unidecode


def formatNumber(num):
    num = float(num)
    if num % 1 == 0:
        return int(num)
    else:
        return num


def fill_nulls_with_none(df):
    """ Fills nulls in a dataframe with None.
        This is required for the Dedupe package to work properly.

        Input: - dataframe with nulls as NaN

        Output: - new dataframe with nulls as None
    """
    new_df = df.copy()
    for col in df.columns:
        new_df[col] = new_df[col].where(new_df[col].notnull(), None)
    return new_df


def fill_col_nulls_with_mean(df, col_name):
    """ Fills nulls in a dataframe with None.
        This is required for the Dedupe package to work properly.

        Input: - dataframe with nulls as NaN

        Output: - new dataframe with nulls as None
    """
    new_df = df.copy()
    new_df[col_name] = new_df[col_name].where(new_df[col_name].notnull(), None)
    return new_df

def convert_numbers_to_strings(df, cols_to_convert, remove_point_zero=True):
    """ Convert number types to strings in a dataframe.
        This is convoluted as need to keep NoneTypes as NoneTypes for what comes next!

        Inputs: - df -> dataframe to convert number types
                - cols_to_convert -> list of columns to convert
                - remove_point_zero -> bool to say whether you want '.0' removed from number

        Ouputs: - dataframe with converted number types
    """
    new_df = df.copy()
    for col in cols_to_convert:
        if remove_point_zero:
            new_df[col] = new_df[col].apply(lambda x: str(x).replace('.0', '') \
                if not isinstance(x, type(None)) else x)
        else:
            new_df[col] = new_df[col].apply(lambda x: str(x) \
                if not isinstance(x, type(None)) else x)
    return new_df


def clean_products_dataset(x_org):
    spacy.cli.download("en_core_web_sm")

    x4_dev = convert_numbers_to_strings(x_org, ['price']).copy(deep=True)
#     x4_dev = x_org.copy(deep=True)
    x4_dev.set_index('instance_id', inplace=True)

    def get_type(record):
        name = record['name'].lower()

        if pd.isna(record['size']):
            if 'tv' in name:
                return 'tv'
            return 'mobile'

        flash_keywords = ['usb', 'drive']
        memory_stick_keywords = ['card', 'stick', 'sd', 'microsd', 'hc', 'class', 'speicherkarte']  # Add variants here

        is_flash = False
        is_memory = False

        for w in flash_keywords:
            if w in name:
                is_flash = True
                break

        for w in memory_stick_keywords:
            if w in name:
                is_memory = True
                break

        if is_flash:
            return 'flash'

        if is_memory:
            return 'stick'

        return 'stick'

    with open('../data/sigmod/translations_lookup_all.json') as fin:
        variants = json.load(fin)

    with open('../data/sigmod/langs_dict.json') as fin:
        json.load(fin)

    # Alpha numeric
    irrelevant_regex = re.compile(r'[^a-z0-9,.\-\s]')
    multispace_regex = re.compile(r'\s\s+')  # Why it doesn't work
    x4_dev.replace({r'[^\x00-\x7F]+': ''}, regex=True, inplace=True)

    for column in x4_dev.columns:
        if column in ['instance_id', 'price']:
            continue
        x4_dev[column] = x4_dev[column].str.lower().str.replace(irrelevant_regex, ' ').str.replace(multispace_regex,
                                                                                                   ' ')

    x4_dev['product_type'] = x4_dev.apply(get_type, axis=1)
#     x4_dev.drop('price', inplace=True, axis=1)
    x4_dev['size'] = x4_dev['size'].str.lower().str.replace(' ', '')
    x4_dev['size'] = x4_dev['size'].where(x4_dev['size'].notnull(), 0)

    # Remove unwanted words from the name
    for i in range(len(x4_dev)):
        record = x4_dev.iloc[i]

        name = record['name']

        # remove unnecessary characters
        basic_punct = '-/\*_,:;/()®™'
        punct_to_space = str.maketrans(basic_punct, ' ' * len(basic_punct))  # map punctuation to space
        name = name.translate(punct_to_space)

        # remove brand
        name = name.replace(record['brand'], '')

        # remove size

        if record.product_type in ['flash', 'stick']:
            name = re.sub('\d\d\d\s?gb', '', name, 6)
            name = re.sub('\d\d\s?gb', '', name, 6)
            name = re.sub('\d\s?gb', '', name, 6)

        tokens = name.split(' ')
        for wd, wdtl in variants.items():
            while wd in tokens:
                tokens.remove(wd)
            for wdt in wdtl:
                while wdt in tokens:
                    tokens.remove(wdt)

        unneeded_words = ['mmoire', 'speicherkarte', 'flashgeheugenkaart', 'flash', 'stick', 'speed', 'high']
        for w in unneeded_words:
            while w in tokens:
                tokens.remove(w)
        x4_dev.iloc[i]['name'] = ' '.join(tokens)

    for column in x4_dev.columns:
        if column in ['instance_id', 'price']:
            continue
        x4_dev[column] = x4_dev[column].str.lower().str.replace(irrelevant_regex, ' ').str.replace(multispace_regex,
                                                                                                   ' ')

    return x4_dev

In [3]:
x4 = pd.read_csv('../data/sigmod/X4.csv')
x4_dev = clean_products_dataset(x4)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


/Users/abdeas0a/opt/anaconda3/envs/sandbox/lib/python3.7/site-packages/ipykernel_launcher.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
x4_dev.head()

,name,price,brand,size,product_type
instance_id,,,,,
altosight.com//0,lexar 32gb 1400x 210mb s professional xqd memr...,31990,lexar,32gb,stick
altosight.com//25,sony microsdxc uhs-1 u3 128gb memriakrtya adap...,59990,sony,128gb,stick
altosight.com//66,"sandisk dual drive, type-c, usb 3.0, 16gb, 130...",6790,sandisk,16gb,flash
altosight.com//68,"sandisk dual drive, type-c, usb 3.0, 64gb, 150...",14190,sandisk,64gb,flash
altosight.com//94,lexar xqd 32gb x1400 professional xqd kupon pr...,329,lexar,32gb,stick


In [5]:
indexer = rl.Index()

In [6]:
indexer.add(rl.index.Block('product_type'))
indexer.add(rl.index.Block('brand'))
indexer.add(rl.index.Block('size'))
# indexer.add(rl.index.SortedNeighbourhood('price', window=3))

In [7]:
candidate_links = indexer.index(x4_dev)

In [8]:
print (len(x4_dev), len(candidate_links))

835 205827


In [56]:
# This cell can take some time to compute.
compare_cl = rl.Compare(n_jobs=-1)

compare_cl.exact('brand', 'brand')
compare_cl.exact('product_type', 'product_type')
compare_cl.exact('size', 'size')
compare_cl.string('price', 'price')
compare_cl.string('name', 'name', method='qgram')
compare_cl.string('name', 'name', method='damerau_levenshtein')
compare_cl.string('name', 'name', method='levenshtein')
# compare_cl.string('name', 'name', method='smith_waterman')
compare_cl.string('name', 'name', method='jarowinkler')
compare_cl.string('name', 'name', method='lcs')  # ---> Takes a lot of time but it gives high boost in the performance

<Compare>

In [57]:
features = compare_cl.compute(candidate_links, x4_dev)

In [58]:
features

0  1  2         3         4  \
instance_id_1        instance_id_2                                      
altosight.com//10033 altosight.com//0     0  1  0  0.333333  0.272727   
                     altosight.com//1129  1  1  0  0.166667  0.290909   
                     altosight.com//1160  0  1  0  0.500000  0.127273   
                     altosight.com//1176  0  1  0  0.500000  0.218182   
                     altosight.com//1178  0  1  0  0.833333  0.200000   
...                                      .. .. ..       ...       ...   
altosight.com//994   altosight.com//909   0  1  0  0.400000  0.125000   
                     altosight.com//911   0  1  0  0.600000  0.186667   
                     altosight.com//926   0  1  0  0.400000  0.157895   
                     altosight.com//94    0  1  0  0.200000  0.111111   
                     altosight.com//95    0  1  0  0.200000  0.111111   

                                                 5         6         7  \
instance_id_1        instance_id_2                                       
altosight.com//10033 altosight.com//0     0.129630  0.129630  0.641280   
                     altosight.com//1129  0.222222  0.222222  0.648003   
                     altosight.com//1160  0.259259  0.259259  0.546296   
                     altosight.com//1176  0.314815  0.314815  0.585053   
                     altosight.com//1178  0.222222  0.222222  0.551363   
...                                            ...       ...       ...   
altosight.com//994   altosight.com//909   0.183908  0.183908  0.534344   
                     altosight.com//911   0.189189  0.189189  0.556480   
                     altosight.com//926   0.243243  0.243243  0.615154   
                     altosight.com//94    0.212500  0.212500  0.577497   
                     altosight.com//95    0.212500  0.212500  0.577497   

                                                 8  
instance_id_1        instance_id_2                  
altosight.com//10033 altosight.com//0     0.317757  
                     altosight.com//1129  0.424242  
                     altosight.com//1160  0.230769  
                     altosight.com//1176  0.359551  
                     altosight.com//1178  0.336842  
...                                            ...  
altosight.com//994   altosight.com//909   0.209677  
                     altosight.com//911   0.234234  
                     altosight.com//926   0.317460  
                     altosight.com//94    0.256410  
                     altosight.com//95    0.256410  

[205827 rows x 9 columns]

In [59]:
features.describe()

,0,1,2,3,4,5,6,7,8
count,205827.000000,205827.000000,205827.000000,205827.000000,205827.000000,205827.000000,205827.000000,205827.000000,205827.000000
mean,0.256827,0.787083,0.261487,0.219799,0.238328,0.232540,0.231745,0.608649,0.351148
std,0.436884,0.409370,0.439445,0.252934,0.114217,0.088753,0.088760,0.070612,0.123396
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.006397,0.006397,0.308786,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.163043,0.183099,0.181818,0.566278,0.271429
50%,0.000000,1.000000,0.000000,0.200000,0.219178,0.218182,0.216981,0.603552,0.341040
75%,1.000000,1.000000,1.000000,0.400000,0.292135,0.262295,0.261364,0.640811,0.420000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [60]:
features.head()

0  1  2         3         4  \
instance_id_1        instance_id_2                                      
altosight.com//10033 altosight.com//0     0  1  0  0.333333  0.272727   
                     altosight.com//1129  1  1  0  0.166667  0.290909   
                     altosight.com//1160  0  1  0  0.500000  0.127273   
                     altosight.com//1176  0  1  0  0.500000  0.218182   
                     altosight.com//1178  0  1  0  0.833333  0.200000   

                                                 5         6         7  \
instance_id_1        instance_id_2                                       
altosight.com//10033 altosight.com//0     0.129630  0.129630  0.641280   
                     altosight.com//1129  0.222222  0.222222  0.648003   
                     altosight.com//1160  0.259259  0.259259  0.546296   
                     altosight.com//1176  0.314815  0.314815  0.585053   
                     altosight.com//1178  0.222222  0.222222  0.551363   

                                                 8  
instance_id_1        instance_id_2                  
altosight.com//10033 altosight.com//0     0.317757  
                     altosight.com//1129  0.424242  
                     altosight.com//1160  0.230769  
                     altosight.com//1176  0.359551  
                     altosight.com//1178  0.336842

In [48]:
features.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7], dtype='int64')

In [49]:
# Add dedupe features
from omegaconf import OmegaConf

params = OmegaConf.create()

params.dataset_type = 'products'
params.train_dataset_path = "../../data/sigmod/X4.csv"
params.label_dataset_path = "../data/sigmod/Y4.csv"
params.save_model_path = "trained_x4.json"
params.save_model_setting_path = "trained_x4_settings.json"
params.columns = ['name', 'brand', 'size', 'product_type']
params.training_file = 'tmp_products_train_data.json'

params.sample_size = 1500
params.recall = 0.9
params.blocked_proportion = 0.9
params.num_cores = 14
params.index_predicates = True


In [21]:
# Create training data dict compatibale with deduper
to_dedupe_dict = x4_dev.to_dict(orient='index')

In [22]:
fields = [
        {'field': 'name', 'type': 'Text', 'has missing': False},
        {'field': 'brand', 'type': 'Exact', 'has missing': False},
        {'field': 'size', 'type': 'Exact', 'has missing': False},
        {'field': 'product_type', 'type': 'Exact', 'has missing': False}]

In [23]:
# Create deduper model
print("Creating dedupe model.")
deduper = dedupe.Dedupe(fields, num_cores=params.num_cores)

# Get the label data
y = pd.read_csv(params.label_dataset_path)

trainig_data = {'match': [], 'distinct': []}
match = y[y.label == 1].to_dict(orient='row')
distinct = y[y.label == 0].to_dict(orient='row')
for m in match:
    trainig_data['match'].append((to_dedupe_dict[m['left_instance_id']], to_dedupe_dict[m['right_instance_id']]))
for d in distinct:
    trainig_data['distinct'].append((to_dedupe_dict[d['left_instance_id']], to_dedupe_dict[d['right_instance_id']]))

# Save the training data
with open(params.training_file, 'w') as fout:
    json.dump(trainig_data, fout)

print("Preparing the training data.")
with open(params.training_file) as tf:
    deduper.prepare_training(to_dedupe_dict, training_file=tf,
                             sample_size=params.sample_size,
                             blocked_proportion=params.blocked_proportion)

# Train the model
print("Training the model.")
deduper.train(recall=params.recall, index_predicates=params.index_predicates)


Creating dedupe model.


/Users/abdeas0a/opt/anaconda3/envs/sandbox/lib/python3.7/site-packages/pandas/core/frame.py:1554: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,
/Users/abdeas0a/opt/anaconda3/envs/sandbox/lib/python3.7/site-packages/pandas/core/frame.py:1554: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,
INFO:dedupe.api:reading training from file


Preparing the training data.


INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (wholeFieldPredicate, name), SimplePredicate: (wholeFieldPredicate, size))
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (wholeFieldPredicate, brand), SimplePredicate: (wholeFieldPredicate, size))


Training the model.


INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
/Users/abdeas0a/opt/anaconda3/envs/sandbox/lib/python3.7/site-packages/rlr/lr.py:39: UserWarning: The line-search routine reaches the maximum number of evaluations.
  case_weights, self.alpha))
INFO:rlr.crossvalidation:optimum alpha: 0.000010, score 0.5086469571452727
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (wholeFieldPredicate, brand), SimplePredicate: (wholeFieldPredicate, size))


In [26]:
# Cluster (prediction stage)
# Please see why the accuracy dropped which cleaning has changed
clustered_dupes = deduper.partition(to_dedupe_dict, 0.37)
print('# duplicate sets', len(clustered_dupes))

# We need to find dedupe features

# duplicate sets 105


In [27]:
clustered_dupes

[(('altosight.com//0',
   'altosight.com//11024',
   'altosight.com//11026',
   'altosight.com//240',
   'altosight.com//2534',
   'altosight.com//3076',
   'altosight.com//4054',
   'altosight.com//5257',
   'altosight.com//6380',
   'altosight.com//760',
   'altosight.com//94',
   'altosight.com//95'),
  array([0.52241073, 0.52481431, 0.52481431, 0.43561523, 0.49118865,
         0.46779111, 0.50668631, 0.32393343, 0.53512944, 0.51322642,
         0.49181101, 0.49181101])),
 (('altosight.com//11319',
   'altosight.com//11422',
   'altosight.com//12629',
   'altosight.com//12634',
   'altosight.com//12635',
   'altosight.com//1286',
   'altosight.com//13248',
   'altosight.com//13250',
   'altosight.com//13259',
   'altosight.com//13814',
   'altosight.com//1444',
   'altosight.com//1447',
   'altosight.com//1543',
   'altosight.com//2123',
   'altosight.com//2425',
   'altosight.com//3644',
   'altosight.com//3686',
   'altosight.com//3698',
   'altosight.com//3978',
   'altosight.com

In [50]:
features['xy_same_entity'] = pd.Series(np.zeros(len(features)))
features.xy_same_entity = 0.0

In [51]:
# Save the result
for el in clustered_dupes:
    for i in range(len(el[0])):
        for j in range(i + 1, len(el[0])):
            k = (el[0][i], el[0][j])
            r_k = (el[0][j], el[0][i])
            p = el[1][i] * el[1][j]

            if k in features.index:
                features.loc[k, 'xy_same_entity'] = p
            
            if r_k in features.index:
                features.loc[r_k, 'xy_same_entity'] = p

In [52]:
features.xy_same_entity.sum()

1702.0533764921554

In [31]:
# Create the labels
gt_lbs = pd.read_csv('../data/sigmod/Y4.csv')

ls = np.zeros(len(features))
labels = features.copy(deep=True)
labels['label'] = ls

len(gt_lbs)

348195

In [32]:
match = []
mismatch = []

for i in range(len(gt_lbs)):
    r = gt_lbs.iloc[i]
    if r.label == 1:
        match.append((r.left_instance_id, r.right_instance_id))
        
    else:
        mismatch.append((r.left_instance_id, r.right_instance_id))

In [33]:
counted = 0
true_inds = []

for el in match:
    counted += (el in features.index)
    counted += ( (el[1], el[0]) in features.index)
    
    if (el in features.index):
        true_inds.append(el)
    if ( (el[1], el[0]) in features.index):
        true_inds.append((el[1], el[0]))

print (counted)
print(len(true_inds))

4082
4082


In [34]:
counted = 0
false_inds = []

for el in mismatch:
    counted += (el in features.index)
    counted += ( (el[1], el[0]) in features.index)
    
    if (el in features.index):
        false_inds.append(el)
    if ( (el[1], el[0]) in features.index):
        false_inds.append((el[1], el[0]))

print (counted)
print(len(false_inds))

201745
201745


In [35]:
labels.loc[true_inds, 'label'] = 1

In [36]:
labels = labels['label']

In [37]:
labels.head(20)

instance_id_1         instance_id_2      
altosight.com//10033  altosight.com//0       0.0
                      altosight.com//1129    0.0
                      altosight.com//1160    0.0
                      altosight.com//1176    0.0
                      altosight.com//1178    0.0
                      altosight.com//1187    0.0
                      altosight.com//1205    0.0
                      altosight.com//1245    0.0
                      altosight.com//1292    0.0
                      altosight.com//1295    0.0
                      altosight.com//1314    0.0
                      altosight.com//1337    0.0
                      altosight.com//1342    0.0
                      altosight.com//1343    0.0
                      altosight.com//1345    0.0
                      altosight.com//1352    0.0
                      altosight.com//1353    0.0
                      altosight.com//1354    0.0
                      altosight.com//1370    0.0
                      altos

In [40]:
features.head()

0  1  2         3         4  \
instance_id_1        instance_id_2                                      
altosight.com//10033 altosight.com//0     0  1  0  0.333333  0.272727   
                     altosight.com//1129  1  1  0  0.166667  0.290909   
                     altosight.com//1160  0  1  0  0.500000  0.127273   
                     altosight.com//1176  0  1  0  0.500000  0.218182   
                     altosight.com//1178  0  1  0  0.833333  0.200000   

                                                 5         6  xy_same_entity  
instance_id_1        instance_id_2                                            
altosight.com//10033 altosight.com//0     0.129630  0.129630             0.0  
                     altosight.com//1129  0.222222  0.222222             0.0  
                     altosight.com//1160  0.259259  0.259259             0.0  
                     altosight.com//1176  0.314815  0.314815             0.0  
                     altosight.com//1178  0.222222  0.222222             0.0

In [38]:
# Lib lightgbm
import lightgbm as lgb

In [53]:
def f1_metric(preds, train_data):

    labels = train_data.get_label()

    return 'f1', f1_score(labels, preds, average='weighted'), True


from sklearn.metrics import f1_score

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True


In [54]:
df_train, labels = features, labels

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
#     'metric': {'binary_logloss'},
#     'num_leaves': 96,
#     'max_depth': 10,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.95,
    'bagging_freq': 5
}
ROUNDS = 200
evals_result = {}

# Features to be trained on.
features_to_train_on = [0, 1, 2, 3, 4, 5, 6, 7, 'xy_same_entity']

# Preparing the input for the LightGBM model.
d_train = lgb.Dataset(df_train,
                      label=labels)

bst = lgb.train(params, d_train, ROUNDS, feval=lgb_f1_score, evals_result=evals_result)

[LightGBM] [Info] Number of positive: 4082, number of negative: 201745
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005119 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1294
[LightGBM] [Info] Number of data points in the train set: 205827, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.019832 -> initscore=-3.900417
[LightGBM] [Info] Start training from score -3.900417


In [55]:
lgb_f1_score(bst.predict(df_train), d_train) # Non dedupe 

('f1', 0.8451217923668101, True)

In [124]:
lgb_f1_score(bst.predict(df_train), d_train) # With dedupe but the bad cleaning stage

('f1', 0.8927231807951987, True)

In [71]:
# Classification step
matches = features[features.sum(axis=1) > 3]
print(len(matches))

438


In [76]:
# Unsupervised learning
# ecm = rl.ECMClassifier(max_iter=1000, binarize=0.8)
# ecm.fit_predict(features)

INFO:root:ECM algorithm converged after 108 iterations


MultiIndex([('www.vology.com//1102',         'buy.net//370'),
            ('www.vology.com//1102',  'www.vology.com//554'),
            ('www.vology.com//1102', 'www.vology.com//4339'),
            ('www.vology.com//1102', 'www.vology.com//1677'),
            ('www.vology.com//1102',  'www.vology.com//668'),
            ('www.vology.com//1102', 'www.vology.com//3032'),
            ('www.vology.com//1102', 'www.vology.com//1430'),
            ('www.vology.com//1102', 'www.vology.com//3957'),
            ('www.vology.com//1102',   'www.vology.com//81'),
            ('www.vology.com//1102', 'www.vology.com//4190'),
            ...
            ( 'www.vology.com//558', 'www.vology.com//1102'),
            ( 'www.vology.com//558', 'www.vology.com//3889'),
            ('www.vology.com//3605', 'www.vology.com//1102'),
            ('www.vology.com//3605', 'www.vology.com//3889'),
            ( 'www.vology.com//756', 'www.vology.com//1102'),
            ( 'www.vology.com//756', 'www.vology.com//

In [130]:
bst.save_model('x4_lgb_classifier.txt', num_iteration=bst.best_iteration) 